In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('sol').getOrCreate()

22/12/23 12:44:34 WARN Utils: Your hostname, Sayans-Macbook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.101 instead (on interface en0)
22/12/23 12:44:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/23 12:44:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/23 12:44:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/12/23 12:44:37 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df = spark.read.csv('walmart_stock.csv', inferSchema=True, header=True)

In [4]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000

22/12/23 13:07:55 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 985648 ms exceeds timeout 120000 ms
22/12/23 13:07:55 WARN SparkContext: Killing executors is not supported by current scheduler.


In [6]:
df.write.format("com.databricks.spark.csv").option("header","true").option("delimiter", "@~").mode("overwrite").save("/Users/sayanadhikary/Desktop/Development/pySpark_1/processed")

In [8]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [9]:
from pyspark.sql.functions import format_number
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [13]:
result = df.describe()

In [19]:
result.select(result['summary'], format_number(result['Open'].cast('float'),2).alias('Value')).show()

+-------+--------+
|summary|   Value|
+-------+--------+
|  count|1,258.00|
|   mean|   72.36|
| stddev|    6.77|
|    min|   56.39|
|    max|   90.80|
+-------+--------+



In [20]:
df.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000

In [24]:
df.withColumn("HV Ratio", df['High']/df['Volume']).select("HV Ratio").show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



In [33]:
peak = df.agg({'High': 'max'})
df.orderBy(df['High'].desc()).head(1)[0][0]

datetime.datetime(2015, 1, 13, 0, 0)

In [40]:
from pyspark.sql.functions import mean
df.agg({'Close':'mean'}).show() #df.select(mean['Close'])

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [43]:
from pyspark.sql.functions import max, min
df.select(max('Volume'), min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [45]:
df.filter('Close < 60').count()

81

In [47]:
#Same as above
df.filter(df['Close'] < 60).count()

81

In [53]:
df.filter(df['High'] > 80).count()/df.count()*100

9.141494435612083

In [57]:
#Pearson correlation
from pyspark.sql.functions import corr
df.select(corr(df['High'], df['Volume']).alias('Pearson Correlation')).show()

+-------------------+
|Pearson Correlation|
+-------------------+
|-0.3384326061737161|
+-------------------+



In [61]:
from pyspark.sql.functions import year
yeardf = df.withColumn("YEAR",year(df['Date'])).select("YEAR", "High")

In [62]:
yeardf.show()

+----+------------------+
|YEAR|              High|
+----+------------------+
|2012|         61.060001|
|2012|         60.349998|
|2012|         59.619999|
|2012|         59.450001|
|2012|         59.549999|
|2012|59.709998999999996|
|2012|         59.529999|
|2012|              60.0|
|2012|59.610001000000004|
|2012|60.110001000000004|
|2012|         60.029999|
|2012|             60.73|
|2012|             61.25|
|2012|             60.98|
|2012|              62.0|
|2012|61.610001000000004|
|2012|             61.84|
|2012|         61.119999|
|2012|             61.32|
|2012|             61.57|
+----+------------------+
only showing top 20 rows



In [65]:
yeardf.groupBy("YEAR").max("High").orderBy("YEAR").show()

+----+---------+
|YEAR|max(High)|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



In [67]:
from pyspark.sql.functions import month
monthdf = df.withColumn("Month", month(df['Date'])).select('Month', 'Close')

In [70]:
monthdf.groupBy('Month').mean('Close').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+

